# Final Project - Piano Roll Extraction -SRC
- Course: AAI-511-Neural Networks
- Institution: University of San Diego
- Professor: Kahila Mokhtari Jadid
- Group 4 Members: 
    * Lucas Young
    * Titouan Margret
    * Juan Pablo Triana Martinez

This notebook has the intention to follow `data_piano_roll_extraction.ipynb`, but using `src` folder with all functions and classes

The following document contains the following:
1. Download *Flavia Dataset* from USD course assignment, else message the zip file is not present.
2. Extract all folders containing Images and labels from `M4-archive.zip`.
3. Loading dataset into `torch.Tensor` and `pandas.DataFrame` when necessary.
4. Pre-processing and cleaning of the data with necessary transformations -> (fized size, grayscale)
5. Split the data into `train_set` and `test_set` using an 80:20 ratio.
6. We would set up a `train_dataset` and `test_dataset` using `torch.utils.data.Dataset` and `torchvision.transforms`
7. We would set up a `train_dataloader` and `test_dataloader` using `torch.utils.data.Dataloader` with Batch size = 32.
8. Setup a `torch.nn` CNN model with the following characteristics:
    - Optimizer: Adam with lr = 0.001
    - Loss function: Cross Entrpy Loss.
    - Same number of layers, we would change the 
9. Plot the `train_loss` and `test_loss`, combined with proper metrics: (Accuracy, Precision, Recall, F1-score)
10. Using that trained model, visualize the learned features across different 